In [3]:
from lib.controller import Controller
from lib.automation import FileDetector
import time
import os

In [4]:
# Commonly changed settings

new_rig_settings = True
datadir = "./tsm_targets/05-31-22" # All files in this directory + subdirectories are loaded
if new_rig_settings:
    datadir = None # auto selects "C:/Turbo-SM/SMDATA/John/mm-dd-yy" on new rig
n_group_by_trials = 5 # Treats every n selected files as trials to combine into single ZDA file

c = Controller(camera_program=4,
              new_rig_settings=new_rig_settings,
              should_auto_launch=False,  # set to False as a safety to avoid double-launch
              datadir=datadir)
print("Data exchange directory:", c.get_data_dir())

Data exchange directory: C:/Turbo-SM/SMDATA/John/06-14-22


In [ ]:
c.start_up() # launch and orchestrate
slice_no = 3
location_no = 1
recording_no = 1
detection_loops = 1

In [ ]:
# auto mode: TSM scheduling
c = Controller(camera_program=4,
              new_rig_settings=new_rig_settings,
              should_auto_launch=False,
              datadir=datadir,  
              filename_PhotoZ_format=True)  # SS_LL_RR.zda naming
c.run_recording_schedule(number_of_recordings=1)  # default: takes 5 trials at 15-s intervals

In [ ]:
new_files = []
# archive directory
dst_dir = c.get_data_dir() + "/" + str(slice_no) + "_" + str(location_no)
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)
    
fd = FileDetector(directory=c.get_data_dir())
for i in range(detection_loops):
    time.sleep(3)
    fd.detect_files()
    new_files += fd.get_unprocessed_file_list()
    if len(new_files) >= n_group_by_trials:
        new_files.sort()
        print("Preparing to process into ZDA file(s)... ")
        time.sleep(5)  # wait for TurboSM to safely finish writing to disk
        
        # process new files
        n_process = len(new_files) - (len(new_files) % n_group_by_trials)
        c.select_files(selected_filenames=new_files[:n_process],
                       slice_no=slice_no,
                       location_no=location_no,
                       recording_no=recording_no)
        c.process_files(n_group_by_trials=n_group_by_trials)
        recording_no += int(len(new_files) / n_group_by_trials)
        
        # auto-archive processed files by moving them to a directory 'slice-loc'
        for pro_file in new_files[:n_process]:
            stripped_file = pro_file.split("/")[-1]
            dst_file = dst_dir + "/" + stripped_file
            os.rename(pro_file, dst_file)
            # tbn file
            os.rename(pro_file[:-4] + ".tbn", dst_file[:-4] + ".tbn")

        
        new_files = new_files[n_process:]
    print("Next recording_no:", recording_no)
    print(new_files)


In [ ]:
# manual mode example: processing only
""" c.select_files(slice_no=3, 
                   location_no=1,
                   recording_no=1, 
                   filename_base="3-1-",
                   filename_start_no=1,
                   filename_end_no=25)
c.process_files(n_group_by_trials=n_group_by_trials) """